In [1]:
import pandas as pd
from google.colab import drive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
googleDrive = GoogleDrive(gauth)
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
Colnames = ['Json_Id', 'Label', 'Statement', 'Subject', 'SpeakerName',
           'JobTitle', 'StateInfo', 'partyAffiliation',
           'BarelyTrueCounts', 'FalseCounts','HalfTrueCounts',
           'MostlyTrueCounts','PantsOnFireCounts','Context']

valid_link = "https://drive.google.com/open?id=1gaS9R-PQvER5DE1dAgh7z6v4u_euIvrO"
fluff, id3 = valid_link.split('=')
valid_filename = 'valid.tsv'
downloaded = googleDrive.CreateFile({'id':id3}) 
downloaded.GetContentFile(valid_filename)
valid_news = pd.read_csv(valid_filename, sep='\t', names = Colnames,error_bad_lines=False)

In [0]:
Factors = [
           {
               "name" : "Political_Affiliation",
               "git" : "https://github.com/jusimon/AlternusVera",
               "folder" :"AlternusVera",
               "file" : "Political_Affiliation",
               "model" : "/content/gdrive/My Drive/Drifters/Models/Politcal_Affiliation_Model.pkl"
               }   
           ]

In [4]:
!rm -rf AlternusVera
!git clone https://github.com/jusimon/AlternusVera

Cloning into 'AlternusVera'...
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 117 (delta 18), reused 18 (delta 6), pack-reused 79
Receiving objects: 100% (117/117), 34.59 MiB | 19.71 MiB/s, done.
Resolving deltas: 100% (52/52), done.


In [5]:
!ls AlternusVera/

AiternusVera_Iteration2a.ipynb			 Common
AlternusVera_Iteration1.ipynb			 input_data
AlternusVera_polical_affiliation_analysis.ipynb  Political_Affiliation.py
AlternusVera_Political_Affiliation2.ipynb	 README.md


In [0]:
!cp AlternusVera/Political_Affiliation.py ./

In [7]:
!ls

adc.json      gdrive			sample_data
AlternusVera  Political_Affiliation.py	valid.tsv


In [0]:
import Political_Affiliation

##### This function calculates fakeness percentage using the weighted_percentage function passing all factors fakeness score. It calls respective factors class to get the fakeness score of individual factor

In [0]:
from gensim.models.doc2vec import Doc2Vec
def fakecheck(dataFrame):
  # biasedFactor = Biased_Factor(dataFrame).predict()
  PoliticalAff = Political_Affiliation.Political_Affiliation(valid_news).predict()
  #  Authenticity = Authenticity(dataFrame).predict()
  #  Sensationalism = Sensationalism(dataFrame).predict()
  # print('sensationalism_factor:',sensationalism_factor)    
  print('political_affilication_factor:',PoliticalAff)
  #  print('spelling_error_factor:',spelling_error_factor)
  #  print('spam_detection_factor:',spam_detection_factor)
  #  print('stance_detection_factor:',stance_detection_factor)
  return weighted_percentage(PoliticalAff, 0, 0, 0)

In [0]:
def weighted_percentage(SensationalismF, politicalAffilitaionF, BiasedF, AuthenticityF):                       
    SensationalismF_Weigth=.84
    BiasedF_Weight=.53
    politicalAffilitaionF_Weight=.65
    AuthenticityF_Weight=.75    
    total_weight=SensationalismF_Weigth+BiasedF_Weight+politicalAffilitaionF_Weight+AuthenticityF_Weight
    return round((((SensationalismF*SensationalismF_Weigth)+
            (politicalAffilitaionF*politicalAffilitaionF_Weight)+
            (BiasedF*BiasedF_Weight)+
            (AuthenticityF*AuthenticityF_Weight))/total_weight)*100,2)

In [11]:
print(weighted_percentage(1,1,1,1))
print(weighted_percentage(0,0,0,0))
print(weighted_percentage(.8,.6,1,.7))

100.0
0.0
76.43


#### Check Fakeness of a news article

In [14]:
input_news='Obamacare will provide insurance to all non-U.S. residents, even if they are here illegally.A strong bipartisan majority in the House of Representatives voted to defund Obamacare.07732584351 - Rodger Burns - MSG = We tried to call you re your reply to our sms for a free nokia mobile + free camcorder. Please call now 08000930705 for delivery tomorrow.Amazon Enters eBook Subscription Market with Kindle Unlimited'
fakecheck(input_news)

political_affilication_factor: 0.5218068535825545


15.82